In [1]:
import re
import pickle
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB
from scipy.sparse import dok_matrix
import numpy as np
nltk.download('stopwords')
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANACLARASAMARINO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
rng = np.random.RandomState(1)
X2 = rng.randint(5, size=(6, 100))

In [3]:
X = dok_matrix((5, 5), dtype=np.int8)

In [4]:
def normaliza_texto_vetorizado(text:str)->str: 
    frase_minuscula = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()
    somente_palavras_validas = re.sub("([^\w]+)|((\w|d)*[\d](\w|d)*)"," ",frase_minuscula)
    texto_normalizado = re.sub("\s+"," ",somente_palavras_validas).strip()
    return texto_normalizado.split(" ")

In [5]:
todos_titulos = pd.read_sql("""
SELECT DISTINCT 
	url_anuncio,	
	titulo_anuncio	
FROM
	anuncios_resumo
""", con, chunksize=1000000)

In [6]:
def constroi_dicionario_palavras(df_pandas_chunked,dicionario):
    qt_linhas_processadas = 0
    for df in df_pandas_chunked:
        qt_linhas_processadas += len(df)
        print(f"qt linhas processadas: {qt_linhas_processadas}")
        for titulo in df['titulo_anuncio']:
            titulo_vetorizado = normaliza_texto_vetorizado(titulo)
            for palavras_titulo in titulo_vetorizado:
                if palavras_titulo:
                    if dicionario.get(palavras_titulo):
                        dicionario[palavras_titulo]+=1
                    else:
                        dicionario[palavras_titulo]=1

In [7]:
def filtrar_dicionario(dicionario_palavras,qtd_minima_palavras_usadas = 5):
    artigos_preposicoes = stopwords.words('portuguese')
    todas_chaves = dicionario_palavras.keys()
    chaves_para_excluir=[]
    for chave in todas_chaves:
        if chave in artigos_preposicoes or dicionario_palavras[chave] < qtd_minima_palavras_usadas or len(chave)<=3:
            chaves_para_excluir.append(chave)
    for chave_ex in chaves_para_excluir:
        del dicionario_palavras[chave_ex]

In [8]:
dicionario_todas_palavras = {}

if not Path('./dicionario_todas_palavras.pkl').is_file():
    constroi_dicionario_palavras(todos_titulos,dicionario_todas_palavras)
    filtrar_dicionario(dicionario_todas_palavras,30)
    with open('./dicionario_todas_palavras.pkl', 'wb') as file:
        pickle.dump(dicionario_todas_palavras, file) 
else :
    with open('./dicionario_todas_palavras.pkl', 'rb') as file:
        dicionario_todas_palavras = pickle.load(file)

lista_todas_palavras = [*dicionario_todas_palavras.keys()]      

In [9]:
df_todas_categorias = pd.read_sql("""
SELECT DISTINCT 
	categoria_atual
FROM
	anuncios_resumo
""", con)
lista_todas_categorias = [*df_todas_categorias['categoria_atual']]

In [10]:
def categoria_para_indice(categoria,lista_todas_categorias):
    if lista_todas_categorias.count(categoria):
        return lista_todas_categorias.index(categoria)
    else:
        return -1
def titulo_para_indices(titulo,lista_todas_palavras):
    lista_palavras_titulo = normaliza_texto_vetorizado(titulo)
    indices_titulo =[]
    for palavra_titulo in lista_palavras_titulo:
        if lista_todas_palavras.count(palavra_titulo):
            indices_titulo.append(lista_todas_palavras.index(palavra_titulo))
    return indices_titulo                 

In [11]:
categoria_para_indice('Artigos infantis',lista_todas_categorias)
titulo_para_indices('baba de cachorro com casa grande',lista_todas_palavras)

[2, 1074, 323, 13]

In [129]:
lista_precos = [*range(10)] # coluna que ira classificar o preço (sera usado log 10)

In [130]:
def preco_para_indice(preco,indice_start=0):
    if isinstance(preco,float) or isinstance(preco,int):
        if preco>1:
            index = np.ceil(np.log10(preco))
            index = int(index) if index>=0 and index<=len(lista_precos) else 0
            return index+indice_start
    return 0

In [131]:
def monta_sql_por_categoria(categoria):
    return f"""SELECT DISTINCT 
                    url_anuncio,
                    titulo_anuncio,
                    categoria_atual,
                    preco_anuncio
                FROM
                    anuncios_resumo
                WHERE categoria_atual = '{categoria}'
                LIMIT 100 -- TIRAR isso DEPOIS
                """

In [132]:
classes_y = [*range(len(lista_todas_categorias))]

In [133]:
clf = BernoulliNB() # Modelo Usado (teste com Bernoulli)

In [134]:
for categoria in lista_todas_categorias:
    df_cat_atual = pd.read_sql(monta_sql_por_categoria(categoria),con)
    print(f'Treinando Modelo para a Categoria: {categoria}')
    qt_colunas = len(lista_todas_palavras)+len(lista_precos)
    X = dok_matrix((len(df_cat_atual), qt_colunas), dtype=np.int8)
    for linha in range(len(df_cat_atual)): # Adiciona valores  na matrix espaça
        indices_palavras_titulo = titulo_para_indices(df_cat_atual['titulo_anuncio'][linha],lista_todas_palavras)
        X[linha,preco_para_indice(df_cat_atual['preco_anuncio'][linha],len(lista_todas_palavras))] = 1 # ref a faixa de precos
        for indice_palavra in indices_palavras_titulo:
            X[linha,indice_palavra] = 1
    Y = np.full(len(df_cat_atual),categoria_para_indice(df_cat_atual['categoria_atual'][0],lista_todas_categorias)) 
    clf.partial_fit(X,Y,classes=classes_y) #Treina o Modelo      

Treinando Modelo para a Categoria: Aluguel - casas e apartamentos
Treinando Modelo para a Categoria: Animais para agropecuária
Treinando Modelo para a Categoria: Antiguidades
Treinando Modelo para a Categoria: Aquários e acessórios
Treinando Modelo para a Categoria: Artigos infantis
Treinando Modelo para a Categoria: Barcos e aeronaves
Treinando Modelo para a Categoria: Beleza e saúde
Treinando Modelo para a Categoria: Bijouterias, relógios e acessórios
Treinando Modelo para a Categoria: Bolsas, malas e mochilas
Treinando Modelo para a Categoria: CDs, DVDs etc
Treinando Modelo para a Categoria: Cachorros
Treinando Modelo para a Categoria: Caminhões
Treinando Modelo para a Categoria: Carros, vans e utilitários
Treinando Modelo para a Categoria: Cavalos
Treinando Modelo para a Categoria: Celulares e telefonia
Treinando Modelo para a Categoria: Ciclismo
Treinando Modelo para a Categoria: Computadores e acessórios
Treinando Modelo para a Categoria: Comércio e indústria
Treinando Modelo par

In [141]:
def procura_categoria(titulo,preco=None,qt_impr=5):
    qt_colunas = len(lista_todas_palavras)+len(lista_precos)
    XN = dok_matrix((1, qt_colunas), dtype=np.int8)
    if preco:
        XN[0,preco_para_indice(preco,len(lista_todas_palavras))] = 1
    indices_palavras_titulo = titulo_para_indices(titulo,lista_todas_palavras)
    for indice_palavra in indices_palavras_titulo:
        XN[0,indice_palavra] = 1
    categorias_prob = clf.predict_proba(XN[0])
    indice_categorias_ordenado = np.flip(np.argsort(categorias_prob))
    limite_impressoes = qt_impr
    for indice_categoria in indice_categorias_ordenado.tolist()[0]:
        # print(indice_categoria)
        print(f'{lista_todas_categorias[indice_categoria]}:{categorias_prob[0][indice_categoria]*100:.2f}%')
        if limite_impressoes<=0:
            break
        limite_impressoes -=1

In [152]:
procura_categoria('honda fit',100000)

Motos:51.88%
Carros, vans e utilitários:17.13%
Barcos e aeronaves:14.36%
Ônibus:4.20%
Caminhões:3.95%
Tratores e máquinas agrícolas:2.42%
